# 任务 4：对话界面 - 与 Llama 3 和 Titan Premier LLM 对话

在此笔记本中，您将使用 Amazon Bedrock 中的 llama3-8b-instruct 和 titan-text-premier 基础模型 (FM) 来构建聊天机器人。

聊天机器人和虚拟助手等对话界面可以增强客户的用户体验。聊天机器人使用自然语言处理 (NLP) 和机器学习算法来理解和响应用户查询。您可以在客户服务、销售和电子商务等各种应用领域中使用聊天机器人，为用户提供快速高效的响应。用户可以通过各种渠道访问聊天机器人，例如网站、社交媒体平台和即时通讯应用程序。

- **聊天机器人（基本）**：采用基础模型的零样本聊天机器人
- **使用提示词模板 (Langchain) 的聊天机器人**：通过提示词模板提供部分上下文的聊天机器人
- **带有角色的聊天机器人**：带有定义的角色的聊天机器人，即职业导师和人类互动
- **上下文感知型聊天机器人**：通过生成嵌入借助外部文件传递上下文。

## 用于通过 Amazon Bedrock 构建聊天机器人的 LangChain 框架

在聊天机器人等对话界面中，记住之前的短期和长期互动变得非常重要。

LangChain 框架提供两种形式的记忆组件。首先，LangChain 提供辅助实用程序，用于管理和操作之前的聊天消息。这些实用程序专为模块化而设计。其次，LangChain 提供将这些实用程序整合到链中的简单方法，使您可以轻松定义不同类型的抽象并与之交互，从而帮助您轻松构建功能强大的聊天机器人。

## 构建上下文感知型聊天机器人 - 关键要素

要构建上下文感知型聊天机器人，第一步便是为上下文生成嵌入。通常，您会执行一个摄取过程，该过程通过您的嵌入模型运行并生成嵌入，然后将嵌入存储在向量存储中。在此笔记本中，您将使用 Titan Embeddings 模型来执行此操作。第二步是用户请求编排、进行交互、调用和返回结果。这包括编排用户请求，与必要的模型/组件进行交互以收集信息，调用聊天机器人来生成响应，然后将聊天机器人的响应返回给用户。

## 任务 4.1：环境设置

在此任务中，您将设置环境。

In [ ]:
#ignore warnings and create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))


In [ ]:
# format instructions into a conversational prompt
from typing import Dict, List

def format_instructions(instructions: List[Dict[str, str]]) -> List[str]:
    """Format instructions where conversation roles must alternate system/user/assistant/user/assistant/..."""
    prompt: List[str] = []
    for instruction in instructions:
        if instruction["role"] == "system":
            prompt.extend(["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n", (instruction["content"]).strip(), " <|eot_id|>"])
        elif instruction["role"] == "user":
            prompt.extend(["<|start_header_id|>user<|end_header_id|>\n", (instruction["content"]).strip(), " <|eot_id|>"])
        else:
            raise ValueError(f"Invalid role: {instruction['role']}. Role must be either 'user' or 'system'.")
    prompt.extend(["<|start_header_id|>assistant<|end_header_id|>\n"])
    return "".join(prompt)

## 任务 4.2：使用 LangChain 的聊天记录开始对话

在此任务中，您将启用聊天机器人，使其能够在与用户的多次互动中传递对话上下文。拥有对话记忆对于聊天机器人来说至关重要，这使聊天机器人能够随着时间的推移进行有意义且连贯的对话。

您可以通过在 LangChain 的 InMemoryChatMessageHistory 类的基础上进行构建来实现对话记忆功能。此对象存储用户与聊天机器人之间的对话，并且聊天机器人代理可以使用历史记录，以便它可以利用先前对话中的上下文。

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i>**注意**：模型输出具有不确定性。

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_aws import ChatBedrock

chat_model=ChatBedrock(
    model_id="meta.llama3-8b-instruct-v1:0" , 
    client=bedrock_client)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer the following questions as best you can."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history = InMemoryChatMessageHistory()


def get_history():
    return history


chain = prompt | chat_model | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key="chat_history",
)
query="how are you?"
response=wrapped_chain.invoke({"input": query})
# Printing history to see the history being built out. 
print(history)
# For the rest of the conversation, the output will only include response

### 新问题

该模型已通过初始消息进行响应。现在，您问它几个问题。

In [ ]:
#new questions
instructions = [{"role": "user", "content": "Give me a few tips on how to start a new garden."}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

### 基于问题进行构建

现在，问一个不提及“garden”这个词的问题，看看模型能否理解之前的对话。

In [ ]:
# build on the questions
instructions = [{"role": "user", "content": "bugs"}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

### 结束此次对话

In [ ]:
# finishing the conversation
instructions = [{"role": "user", "content": "That's all, thank you!"}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

## 任务 4.3：使用提示词模板 (LangChain) 的聊天机器人

在此任务中，您将使用默认的 PromptTemplate 来构建此输入。LangChain 提供多个类和函数，以便轻松构建和使用提示词。

In [ ]:
#  prompt for a conversational agent
def format_prompt(actor:str, input:str):
    formatted_prompt: List[str] = []
    if actor == "system":
        prompt_template="""<|begin_of_text|><|start_header_id|>{actor}<|end_header_id|>\n{input}<|eot_id|>"""
    elif actor == "user":
        prompt_template="""<|start_header_id|>{actor}<|end_header_id|>\n{input}<|eot_id|>"""
    else:
        raise ValueError(f"Invalid role: {actor}. Role must be either 'user' or 'system'.")   
    prompt = PromptTemplate.from_template(prompt_template)     
    formatted_prompt.extend(prompt.format(actor=actor,input=input))
    formatted_prompt.extend(["<|start_header_id|>assistant<|end_header_id|>\n"])
    return "".join(formatted_prompt)

In [ ]:
# chat user experience
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()


    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            with self.out:
                print("Thank you , that was a nice chat !!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        response = self.qa.invoke({"input": prompt})
                        result=response['answer']
                    else:
                        instructions = [{"role": "user", "content": prompt}]
                        #result = self.qa.invoke({'input': format_prompt("user",prompt)}) #, 'history':chat_history})
                        result = self.qa.invoke({"input": format_instructions(instructions)})
                except:
                    result = "No answer"
                thinking.value=""
                print(f"AI:{result}")
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

接下来，开始进行对话。

In [ ]:
# start chat
history = InMemoryChatMessageHistory() #reset chat history
chat = ChatUX(wrapped_chain)
chat.start_chat()

In [ ]:
print(history)

## 任务 4.4：带有角色的聊天机器人

在此任务中，人工智能 (AI) 助手扮演职业教练的角色。角色扮演对话需要在开始对话之前预先填充用户消息。ConversationBufferMemory 用于为对话预填充信息。

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", " You will be acting as a career coach. Your goal is to give career advice to users. For questions that are not career related, don't provide advice. Say, I don't know."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history = InMemoryChatMessageHistory() # reset history

chain = prompt | chat_model | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key="career_chat_history",
)

response=wrapped_chain.invoke({"input": "What are the career options in AI?"})
print(response)

In [ ]:
response=wrapped_chain.invoke({"input": "How to fix my car?"})
print(response)

In [ ]:
print(history)

Now, ask a question that is not within this persona's specialty. The model should not answer that question and should give a reason for that.

## 任务 4.5 上下文感知型聊天机器人

在此任务中，您将要求聊天机器人根据传递给它的上下文来回答问题。您将提供一个 CSV 文件，并使用 Titan Embeddings 模型来创建代表该上下文的向量。该向量存储在 Facebook AI Similarity Search (FAISS) 中。当向聊天机器人提问时，您需要将此向量传递回聊天机器人，并让它使用此向量检索答案。

### Titan Embeddings 模型

嵌入将单词、短语或任何其他离散项目表示为连续向量空间中的向量。这样一来，机器学习模型即可对这些表示形式执行数学运算，并捕获它们之间的语义关系。

您将嵌入用于检索增强生成 (RAG) [文档搜索功能](https://labelbox.com/blog/how-vector-similarity-search-works/)。

In [ ]:
# model configuration
from langchain_aws.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate

br_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=bedrock_client)

#### FAISS 作为 VectorStore

为了使用嵌入进行搜索，您需要一个能够高效执行向量相似度搜索的存储。在此笔记本中，您使用的是 FAISS，它是一种内存中存储。要永久存储向量，您可以使用 Knowledge Bases for Amazon Bedrock、pgVector、Pinecone、Weaviate 或 Chroma。

[此处](https://python.langchain.com/v0.2/docs/integrations/vectorstores/) 提供了 LangChain VectorStore API。

In [ ]:
# vector store
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

loader = CSVLoader("../rag_data/Amazon_SageMaker_FAQs.csv") # --- > 219 docs with 400 chars
documents_aws = loader.load() #
print(f"documents:loaded:size={len(documents_aws)}")

docs = CharacterTextSplitter(chunk_size=2000, chunk_overlap=400, separator=",").split_documents(documents_aws)

print(f"Documents:after split and chunking size={len(docs)}")
vectorstore_faiss_aws = None
try:
    
    vectorstore_faiss_aws = FAISS.from_documents(
        documents=docs,
        embedding = br_embeddings, 
        #**k_args
    )

    print(f"vectorstore_faiss_aws:created={vectorstore_faiss_aws}::")

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

### 运行快速低代码测试 

您可以使用 LangChain 提供的 Wrapper 类来查询向量数据库存储并返回相关文档。这将使用所有默认值运行 QA 链。

In [ ]:
chat_llm=ChatBedrock(
    model_id="amazon.titan-text-premier-v1:0" , 
    client=bedrock_client)
# wrapper store faiss
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_aws)
print(wrapper_store_faiss.query("R in SageMaker", llm=chat_llm))

### 聊天机器人应用程序

对于聊天机器人，您需要上下文管理、历史记录、向量存储和许多其他组件。您首先需要构建一个 ConversationalRetrievalChain。

这使用了 **create_stuff_documents_chain** 和 **create_retrieval_chain** 函数

### RAG 使用的参数和功能

- **Retriever**：您使用了 `VectorStoreRetriever`，它由 `VectorStore` 支持。要检索文本，有两种搜索类型可供选择：`"similarity"` 或 `"mmr"`。`search_type="similarity"` 在检索器对象中使用相似性搜索，它会选择与问题向量最相似的文本块向量。

- **create_stuff_documents_chain** 指定如何将检索到的上下文输入到提示和 LLM 中。检索到的文档被“填充”为上下文，而无需进行任何摘要或其他处理即可进入提示。

- **create_retrieval_chain** 添加检索步骤并通过链传播检索到的上下文，并将其与最终答案一起提供。

如果提出的问题超出了上下文范围，模型就会回答说它不知道答案。

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

retriever=vectorstore_faiss_aws.as_retriever()
question_answer_chain = create_stuff_documents_chain(chat_llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "What is sagemaker?"})
print(response) # shows the document chunks consulted to come up with the answer

接下来。开启一个对话

In [ ]:
chat = ChatUX(rag_chain, retrievalChain=True)
chat.start_chat()  # Only answers will be shown here, and not the citations


您已经使用 Titan LLM 创建了具有以下模式的对话界面：

- 聊天机器人（基本 - 无上下文）
- 使用提示词模板 (Langchain) 的聊天机器人
- 带有角色的聊天机器人
- 上下文感知型聊天机器人

### 自行尝试

- 将提示词更改为您的特定使用案例，并评估不同模型的输出。
- 尝试不同的 Token 长度，了解服务的延迟和响应能力。
- 应用不同的提示词工程原则，获得更好的输出。

### 清理

您已完成此笔记本。要进入本实验的下一部分，请执行以下操作：

- 关闭此笔记本文件并继续执行**任务 5**。